 implementação do problema da mochila para algoritmo genetico, baseado no exemplo de aplicação no livro: 'introduction to evolutionary computing

In [1]:
import random

In [4]:
class KnapSackSolution:
    def __init__(self, genotype, items, capacity):
        self.genotype = genotype
        self.items = items # (value, weight)
        self.capacity = capacity
        
        self.phenotype, self.total_value, self.total_weight = self.decode()
        
    def decode(self):
        #converte o genotipo numa solucao factivel (fenotipo)
        total_weight =0
        total_value = 0
        phenotype = []
        
        for bit, (value, weight) in zip(self.genotype, self.items):
            if bit == 1:
                if total_weight + weight <= self.capacity:
                    phenotype.append((value, weight))
                    total_weight += weight
                    total_value += value
        return phenotype, total_value, total_weight
    
    def fitness(self):
        return self.total_value
    
    def __str__(self):
        return f"Phenotype: {self.phenotype}\nTotal Value: {self.total_value}\nTotal Weight: {self.total_weight}"
    
    def __repr__(self):
        return self.__str__()


In [12]:
#initialize the population of solution objects
def intialize_population(population_size, items, capacity):
    population = []
    for _ in range(population_size):
        genotype = [random.randint(0,1) for _ in range(len(items))]
        population.append(KnapSackSolution(genotype, items, capacity))
    return population

def tournament_selection(population, k):
    individuals = random.sample(population, k)
    return max(individuals, key=lambda x: x.fitness())

# one point crossover
def crossover(parent1, parent2, crossover_rate):
    point = random.randint(0, len(parent1.genotype)-1)
    if(random.randint(0,100) < crossover_rate):
        offspring1 = parent1.genotype[:point] + parent2.genotype[point:]
        offspring2 = parent2.genotype[:point] + parent1.genotype[point:]
        return KnapSackSolution(offspring1, parent1.items, parent1.capacity),KnapSackSolution(offspring2, parent1.items, parent1.capacity)
    else:
        return parent1, parent2
    
def mutation(solution, mutation_rate):
    for i  in range(len(solution.genotype)):
        if(random.randint(0,100) < (mutation_rate*100)):
            solution.genotype[i] = 1 - solution.genotype[i] # flips the bit
    return solution

def genetic_algorithm(items, capacity=10, population_size=500, crossover_rate=70, mutation_rate=0.05, generations=100, max_stagnant_generations=25):
    population = intialize_population(population_size, items, capacity)
    best_solution = max(population, key=lambda x: x.fitness())
    best_fitness = best_solution.fitness()
    
    stagnant_generations = 0
    generation = 0
    
    while True:
        new_population = []
        for _ in range(population_size//2):
            parent1 = tournament_selection(population, 2)
            parent2 = tournament_selection(population, 2)
            offspring1, offspring2 = crossover(parent1, parent2, crossover_rate)
            offspring1 = mutation(offspring1, mutation_rate)
            offspring2 = mutation(offspring2, mutation_rate)
            new_population.append(offspring1)
            if(len(new_population) < population_size):
                new_population.append(offspring2)
        population = new_population
        current_best = max(population, key=lambda x: x.fitness())
        if (current_best.fitness() > best_fitness):
            best_fitness = current_best.fitness()
            best_solution = current_best
            stagnant_generations = 0
        else:
            stagnant_generations +=1
        if(stagnant_generations >= max_stagnant_generations):
            break;
        if(generation >= generations):
            break;
        generation+=1

    return best_solution, best_fitness, generation, stagnant_generations

In [25]:
items = [
    (2000, 3),    # value, weight
    (3000, 4),
    (1500, 2),
    (100, 1),
    (800, 1),
    (2500, 2),
    (1000, 2),
    (1200, 1),
    (300, 1),
    (500, 4)
]
population_size = 500
generations = 100
crossover_rate = 70
mutation_rate = 1/len(items)
bag_capacity = 10

best_solution, best_fitness, generation, stagnant_generations = genetic_algorithm(items, bag_capacity, population_size, crossover_rate, mutation_rate)
print(f"Best Solution: {best_solution}")
print(f"Best Fitness: {best_fitness}")
print(f"Generation : {generation}")
print(f"Stagnant Generations: {stagnant_generations}")



Best Solution: Phenotype: [(3000, 4), (1500, 2), (800, 1), (2500, 2), (1200, 1)]
Total Value: 9000
Total Weight: 10
Best Fitness: 9000
Generation : 24
Stagnant Generations: 25
